## Downloading datasets
First we need to download the datasets and load them into batches. To do this we use PyTorch's DataLoader class.

The dataset used in this implementation is [PascalVOC 2012](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/), which is commonly used in object detection problems. This dataset is XML-based.

The original AlexNet paper uses ImageNet, a very large dataset of labeled high-res images. PascalVOC, similarly to ImageNet, contains variable resolution images. Therefore it is important to down-scale image resolution due to AlexNet's requirement of constant input dimensionality. 

`Therefore, we down-sampled the images to a fixed resolution of 256 × 224. Given a
rectangular image, we first rescaled the image such that the shorter side was of length 224, and then
cropped out the central 224×224 patch from the resulting image. We did not pre-process the images
in any other way, except for subtracting the mean activity over the training set from each pixel. So
we trained our network on the (centered) raw RGB values of the pixels`

In [5]:
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

In [22]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227)),
    transforms.CenterCrop((227, 227)),  # Add CenterCrop to crop to 227x227
])

cifar10_classes = CIFAR10(root='./data', train=True, download=True).classes

# Create CIFAR-10 datasets
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [23]:
for i, class_name in enumerate(cifar10_classes):
    print(f'{i+1}: {class_name}')

1: airplane
2: automobile
3: bird
4: cat
5: deer
6: dog
7: frog
8: horse
9: ship
10: truck


In [24]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
from itertools import islice

num = 5

def show_batch(batch_images):
    fig,ax = plt.subplots(figsize=(16,12))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(batch_images,nrow=16).permute(1,2,0))

# for batch_images, _ in islice(train_loader, num):
#     show_batch(batch_images)

batch = next(iter(train_loader))

print(batch[1])

image = batch[0][0]
image.shape

tensor([4, 2, 2, 1, 4, 5, 8, 5, 1, 3, 7, 0, 8, 7, 2, 2])


/home/jamie/code/personal/alexnet/.env/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([3, 227, 227])

In [25]:
from torch import nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4, padding=0), # in, out, kernel_size
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(256*6*6, 4096), # in, out
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
        )
        self.fc3 = nn.Linear(4096, num_classes)

    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)

        x = x.view(-1, 256*6*6)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

In [26]:
alexnet = AlexNet()

In [27]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimiser = optim.SGD(alexnet.parameters(), momentum=0.9, lr=0.005, weight_decay=0.005)

In [28]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimiser.zero_grad()

        outputs = alexnet(inputs)
        labels = labels.squeeze()

        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        running_loss += loss.item()
        if i % 20 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = alexnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: {:.2%}'.format(correct / total))
